Turns out Microsoft Edge doesn't really work on mac. Who knew. Look into remote driver later

In [327]:
'''
<> On Deck <> Still caught by bot detector, disables point collection after 5 searches
Add in random delay for sleep
consider openning a second tab and duplicating
pick 50 items from dataframe at random
going back to top site and clicking on a news card
to use this you must replace login_password and my email
'''
print()

In [328]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
from random import randint

In [338]:
word_bank = './word_bank.csv'
password = '../../Daka/bing_login/pass.csv'

In [339]:
df = pd.read_csv(word_bank)
login = pd.read_csv(password)
login_pass = login.passwords[0]

In [340]:
# Loading Webpage

# set user agent
opts = Options()
opts.add_argument("user-agent=Tony Alverez")

driver = webdriver.Chrome(chrome_options=opts)

driver.set_window_size(1280, 720)
driver.get('https://www.bing.com/')


In [341]:
# Logging into account
sleep(2)
logger = driver.find_element_by_xpath('//*[@id="id_s"]')
logger.click()

In [342]:
sleep(2)
driver.find_element_by_xpath('//*[@id="i0116"]').clear()
element = driver.find_element_by_xpath('//*//*[@id="i0116"]')
element.send_keys('gnnjume@gmail.com')
element.send_keys(Keys.ENTER)

In [343]:
# every selenium instance restarts mem
# rememeber = driver.find_element_by_xpath('//*[@id="idChkBx_PWD_KMSI0Pwd"]')
# rememeber.click()

In [ ]:
sleep(2)
driver.find_element_by_xpath('//*[@id="i0118"]').clear()
element = driver.find_element_by_xpath('//*[@id="i0118"]')
element.send_keys(login_pass)
element.send_keys(Keys.ENTER)

In [ ]:
# Database search loop
num=0
for i in range(4): # (df.shape[0]):
    x = randint(1,3)
    y = randint(1,7)
    sleep(randint(3,10))
    if num < df.shape[0]:   
        driver.find_element_by_xpath('//*[@id="sb_form_q"]').clear()
        element = driver.find_element_by_xpath('//*[@id="sb_form_q"]')
        element.send_keys(df.posts[num])
        element.send_keys(Keys.ENTER)
        sleep(2)
    else:
        num -= randint(15,30)
    # scroll up and down at random
    for i in range(x): # 3 is end of page results
        scroll = driver.find_element_by_xpath('/html/body')
        scroll.send_keys(Keys.SPACE)
        sleep(randint(1,3))
        if x+2 <= 4:
            scroll.send_keys(Keys.SHIFT, Keys.SPACE)
    # checks out first link
    if y <= 3:
        tab_rand = driver.find_element_by_xpath('/html/body')
        tab_rand.send_keys(Keys.TAB, Keys.ENTER)
        sleep(randint(2,5))
        # repeat scroll def here
        driver.back()
   
    # heads back to bing home
    if y == 5:
        driver.find_element_by_xpath('//*[@id="sb_form"]/a').click()
    
    # addition can be used to make random choice for def start if > df.shape[0] minus random large number
    num += randint(1,30) 

In [ ]:
sleep(2)
driver.close()

In [ ]:
# <li class="b_algo" data-bm="9">
lookup = driver.find_element_by_class_name("b_algo")
lookup.click()

In [ ]:
y = 3
driver.find_element_by_xpath('//*[@id="sb_form_q"]').clear()
element = driver.find_element_by_xpath('//*[@id="sb_form_q"]')
element.send_keys(df.posts[30])
element.send_keys(Keys.ENTER)
sleep(2)
if y == 3:
    tab_rand = driver.find_element_by_xpath('/html/body')
    tab_rand.send_keys(Keys.TAB, Keys.ENTER)